# Density-functional theory

In this notebook we will dive into density-functional theory (DFT).
As I will describe simulating electrons in periodic systems with DFT
is a common setting where the periodic one-particle Hamiltonians $H$
described in the previous notebook arise.

## Motivation and quantum-mechanical setting

- The goal of electronic structure is to describe the behaviour of electrons in matter.
- This allows to understand the chemical and physical properties of materials.




- For accurately predicting properties from first principles a quantum-mechanical description of electrons is required.
- The **states** describing the motion and behaviour of electrons are complex-valued
  functions in an infinite-dimensional function space, which is a Hilbert space.
- The **operators** are mappings defined on this Hilbert space.




- The motion of electrons is highly correlated. Pictorially speaking the motion of one electron depends on the motion of all others.
- This makes an accurate modelling of the electronic structure rather involved.
- In this workshop we focus on **density-functional theory** (DFT).
  While this approximate model has a number of limitations,
  it has become one of the most widely used practical methods due to its balance 
  between cost and accuracy. 
- Moreover problems of the form of DFT are usually the first step in modelling
  the electronic structure, even when a more accurate methods will eventually be
  targeted. So understanding how DFT works represents a good step in either case.
  
**More details**
- My [2020 Juliacon talk](https://www.youtube.com/watch?v=-RomkxjlIcQ) (motivation for electronic-structure theory and DFT)
- Chapter 2 of [my PhD thesis](https://michael-herbst.com/publications/2018.05_phd_corrected.pdf) (Mathematical formulation of quantum mechanics

## Formulation of the DFT problem

- In DFT an electronic state is described by a set of orthonormal **orbitals**
  $\{\psi_i\}$ ($i = 1, 2, \ldots N$).
- Each of these orbitals is a function from $\mathbb{R}^3$ to $\mathbb{C}$.
- To a good approximation one can think of one orbital $\psi_i$ as the description
  of the behaviour of one electron pair.
- If there are $2 N$ electrons in the system, we therefore need $N$ orbitals.

- Of largest physical interest is the ground state of a system, that is the state
  which features the lowest energy.
- At the level of DFT this translates to the minimisation problem
  $$ \displaystyle \text{min}_{\{\psi_i\}} \mathcal{E}_\text{DFT}(\{\psi_i\}) $$
  where we seek the lowest-energy orbitals given a DFT energy functional.


- The details of the energy functional depend on the chosen DFT model.
  The common structure is as follows:

    $$ \mathcal{E}_\text{DFT}(\{\psi_i\}) = \sum_{i=1}^N 2 \int \psi_i^\ast \left(-\frac12 \Delta\right) \psi_i
    + \int V_\text{ext} \rho + \int V_\text{H}[\rho] \rho + \int V_\text{xc}[\rho] \rho + E_\text{nuclear}$$
    
  with
     * the electron **density** $\rho = \sum_i^N |\psi_i|^2$
       being directly dependent on *all* orbitals.
     * $\sum_i  \int \psi_i^\ast \left(-\frac12 \Delta\right) \psi_i$ describing
       the **kinetic** energy of the electrons
       (i.e. the term we discussed in the previous notebook)
     * $\int V_\text{ext} \rho$ being the **external** potential energy,
       i.e. the electron-nuclear interaction.
     * $\int V_\text{H}[\rho] \rho$ being the **Hartree** energy,
       i.e. the classical Coulombic electron-electron repulsion
     * $\int V_\text{xc}[\rho] \rho$ being the **exchange-correlation**
       (or XC) energy, which describes the electron-electron interaction beyond
       the classical model.
     * $E_\text{nuclear}$: A fixed energy offset due to the repulsion of the nuclei
       from each other.

**Note:** The Hartree potential $V_\text{H}$ and the XC potential $V_\text{xc}$ themselves depend on the density $\rho$. Therefore $\mathcal{E}_\text{DFT}$ is non-linear in the orbitals!

For our numerical experiments we will use the [density-functional toolkit](https://dftk.org) (DFTK).
In DFTK such a DFT model can be conveniently assembled by collecting the respective energy terms:

In [ ]:
using DFTK

# Build up the LDA model by assembling the energy terms
terms_LDA = [Kinetic(),           # -1/2 Δ
             AtomicLocal(),       # part of Vₑₓₜ
             AtomicNonlocal(),    # part of Vₑₓₜ
             PspCorrection(),     # part of Vₑₓₜ
             Hartree(),           # Vₕ
             Xc(:lda_xc_teter93), # Vxc
             Ewald()]             # part of Eₙᵤᵪₗₑₐᵣ

DFTK has many more terms to offer (e.g. external fields, magnetism, etc.). See [the source code](https://dftk.org/tree/master/src/terms) for the full list.

## Solving DFT: The self-consistent field problem

The stationarity conditions (Euler-Lagrange equations) corresponding to the DFT minimisation problem are

$$\left\{\begin{aligned}
    &\forall 1 \leq i \leq N: &\left(-\frac12 \Delta + V[\rho] \right) \psi_i &= \varepsilon_i \psi_i
    \qquad \text{(with $\varepsilon_1, \varepsilon_2, \ldots, \varepsilon_N$ smallest)} \\
    &\forall 1 \leq i, j \leq N: &\int \psi_i^\ast \psi_j &= \delta_{ij} \\
    &&V[\rho] &= V_\text{ext} + V_\text{H}[\rho] + V_\text{xc}[\rho] \\
    &&\rho &= \sum_{i=1}^N 2 |\psi_i|^2
\end{aligned}\right.$$

in which $H[\rho] = -\frac12 \Delta + V[\rho]$ is the **DFT Hamiltonian**. These equations are called the **self-consistent field** (SCF) problem (for historic reasons).

- The first and second line are basically an eigenvalue problem
  with the usual orthogonality between the resulting eigenfunctions.
- Since $V$ depends on $\rho$ (3rd line), which itself depends on $\psi_i$ (last line)
  solving the SCF equations implies solving a *non-linear* eigenvalue problem.
- In this sense the DFT Hamiltonian is a straight generalisation of the problems
  we discussed in the previous notebook.

To allow for writing the SCF problem more compact
we define the **potential-to-density map**
$$
D(V) = \sum_{i=1}^N |\psi_i|^2 \qquad \text{$\psi_i$ are the $N$ lowest eigenvectors of $-\frac12 \Delta + V$}.
$$
With it the SCF problem can be written as
$$ \rho = D(V(\rho)). $$
Clearly an SCF is nothing else but a **fixed-point problem**.

**Note:** In the picture I sketch in this notebook I make a few assumptions for simplicity (e.g. no unpaired electrons, no spin, only integer occupation, existence of a gap, etc.). Adding in these extra complications does not change much of the bottom line, so I've decided to leave these aspects out for clarity. You can find more details in the references below.

#### More details
- More references on DFT (at various levels) are given in the [DFTK documentation](https://juliamolsim.github.io/DFTK.jl/stable/guide/density_functional_theory/)

## Modelling silicon

Puh ... that was quite a lot of equations. Let's see some numbers!

First we will use the LDA model defined above to model silicon,
a nicely periodic lattice.

| <img src="img/Silicon_crystal.jpg" width=180 height=180 /> |
| --------------- |
| silicon crystal |

In [ ]:
using DFTK

# Look up the structure of silicon in a book
# and define the lattice and the atomic positions
a = 10.26
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]]

# Setup a model in DFTK from the terms we defined above
model = Model(lattice; atoms, terms=terms_LDA)
# Note: For common models DFTK has shortcuts. In this case
#            model = model_LDA(lattice, atoms)
#       does exactly the same

# Discretise the problem in a plane-wave basis
# We use a mesh of 2x2x2 k-Points in the Brillouin zone
# and an energy cutoff of 13 Hartree
basis  = PlaneWaveBasis(model; Ecut=15, kgrid=[2, 2, 2])

# Run the SCF and catch the result:
scfres = self_consistent_field(basis);

Ok so this did something, but what?

- First DFTK inspects the molecule to guess an initial density $\rho_0$.
- Then we solve $ \rho = D(V(\rho)) $ by computing
  $D(V(\rho_n))$ for a sequence of iterates $\rho_n$ until input and output
  are close enough, then we flag convergence.
- We will discuss these algorithms with more details in the next notebook.

In the end we obtained the ground-state energy of silicon as

In [ ]:
scfres.energies

We can also look at the density:

In [ ]:
using Plots
heatmap(scfres.ρ[:, :, 5], c=:Blues)

... or access the eigenpairs of the final Hamiltonian diagonalisation
in `scfres.ψ` and `scfres.eigenvalues`:

In [ ]:
scfres.eigenvalues

It might come to a surprise for your why this does not return a flat list,
but in fact a list of two lists. This will be clarified in the following


**Remark:** Since silicon (and in fact all problems we will consider) are periodic,
the DFT Hamiltonian $H$ is also periodic.
Therefore we are in the periodic setting discussed in the previous notebook
and one is able to apply the Bloch transform to $H$.
This allows to reformulate the SCF problem as 

$$\left\{\begin{aligned}
    &\forall k\in\text{BZ}, \ \  1 \leq i \leq N: &\left(\frac12 (-i \nabla + k)^2 + V[\rho] \right) \psi_{ki} &= \varepsilon_{ki} \psi_{ki}\\
    &\forall k\in\text{BZ}, \ \ 1 \leq i, j \leq N: &\int \psi_{ki}^\ast \psi_{kj} &= \delta_{ij} \\
    &&V[\rho] &= V_\text{ext} + V_\text{H}[\rho] + V_\text{xc}[\rho] \\
    &&\rho &= \sum_{k\in\text{BZ}} \sum_{i=1}^N 2 |\psi_{ki}|^2
\end{aligned}\right.$$

In other words we can diagonalisate $H$ $k$-Point by $k$-Point
and only at the level of computing the density we need to
sum over $k$-Points.

In DFTK most quantities in the `scfres` are therefore stored
as a list over $k$-Points. Therefore
```julia
    scfres.eigenvalues[1]
```
gives access to the $\varepsilon_{ki}$ for the first $k$-Point.

In our problem we used `kgrid=[2,2,2]`. One would therefore expect 8 $k$-Points
to be present in the discretisation. As it turns out there are only

In [ ]:
length(basis.kpoints)

The reason for this is that the symmetries of the problem allow to make further reductions in effort. If we disable symmetries, we get the expected 8 $k$-Points:

In [ ]:
model_nosym = Model(lattice; atoms, terms=terms_LDA, symmetries=false)
length(PlaneWaveBasis(model_nosym; Ecut=15, kgrid=(2, 2, 2)).kpoints)

To conclude this section, let us plot the band structure of silicon,
i.e. the relationship between the eigenvalues of $H$
and the position $k$ inside the Brillouin zone.
(For more details see the previous notebook).

In [ ]:
plot_bandstructure(scfres)

## Modelling aluminum

**Exercise:** Try running a simulation of aluminium yourself.

For aluminum a possible structural setup is
```julia
a = 7.65339
lattice = diagm(fill(a, 3))
Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
atoms = [Al => [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]]
```

First use the same LDA model as before, defined using the DFTK shortcut
```julia
model = model_LDA(lattice, atoms)
```
and use `Ecut=15` and `kgrid=[2, 2, 2]` to discretise the problem.

When running the SCF like this you should observe either slow or no convergence.
This points to the fact that modelling aluminium is clearly a little different
than silicon.
In fact the issue is due to aluminium being a metal,
while silicon is an insulator / semiconductor.
More on this below.

To make progress and achieve SCF convergence add the parameter
```julia
model = model_LDA(lattice, atoms, temperature=1e-3)
```
This is called **smearing** and is a numerical trick
to improve the convergence for quantities which involve
sums over $k$-Points in metals.
As the density $\rho$ involves such a sum, the SCF typically fails
to converge in metals without smearing.

With this setup it the SCF should succeed. Use the obtained scfres to compute the band structure of aluminium with `plot_bandstructure(scfres)`.

In [ ]:
using DFTK

a = 7.65339
lattice = diagm(fill(a, 3))
Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
atoms = [Al => [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]]

# your solution here 

## Contrasting aluminium and silicon

With these two calculations we have computed two band structures,
one for a metal (aluminium) and one for a semiconductor (silicon).

- All details of the band structure do not matter for our purposes.
- The important difference is that silicon has a so-called **band gap**,
  i.e. a region near the Fermi level (which is at $0$ in the shown band plots)
  where there are no eigenvalues, i.e. no blue bands.
  In contrast aluminium does not have this gap.
- This makes aluminium a metal and silicon an insulator respectively semiconductor.

For the numerical treatment the absence of a band gap has strong implications:
- At the beginning of the SCF the DFT Hamiltonian $H[\rho_n]$
  at the current density $\rho_n$ is only a rough approximation to the
  converged Hamiltonian $H[\rho_\ast]$.
- This means that the order of the $\psi_{ki}$ near the Fermi level,
  i.e. near $i = N$, may be easily swapped as many $\psi_{ki}$
  of similar eigenvalues $\varepsilon_i$ are close.
- From one iteration to the next the $\psi_{ki}$ which are summed
  over in the density expression
  $$ \rho = \sum_{k\in\text{BZ}} \sum_{i=1}^N 2 |\psi_{ki}|^2 $$
  may thus change (as $N$ becomes $N+1$, thus not summed over, but $N+1$ becomes $N$).
- Without going into details the smearing temperature reduces this
  problem by not enforcing a hard cut between the functions summed over in the density,
  but employing a smoother cutoff function.

- As becomes apparent already in these first few experiments
  the numerics of an SCF and the physics of the system are closely linked.


#### Takeaways
- Insulators / semiconductors have a band gap
- Metals have no band gap
- Without a band gap a smearing is needed to aid convergence.
- SCF convergence properties for metals and insulators differ quite drastically (more on this in the next notebooks)